## RL agent Q-learning for TicTacToe env

The [Tic-Tac-Toe](https://github.com/MauroLuzzatto/OpenAI-Gym-TicTacToe-Environment) is a simple game environment that allows to train reinforcement learning agents. These notebook contains an implemetation of Q-learning with epsilon-greedy strategy for TicTacToe env.

In [22]:
# load the python modules
import time
import sys
import warnings

import gym
import numpy as np
from tqdm import tqdm
import gym_TicTacToe

from src.qagent import QLearningAgent
from src.play_tictactoe import play_tictactoe, play_tictactoe_with_random

from src.utils import (
    create_state_dictionary,
    reshape_state,
    save_qtable,
    load_qtable
)

# ignore warnings
if not sys.warnoptions:
    warnings.simplefilter("ignore")

In [23]:
class Player:
    def __init__(self, color, episodes: int):
        self.color = color
        self.reward_array = np.zeros(episodes)
        self.name = f"Player {color}"

In [24]:
# initialize the tictactoe environment

REWARD_LARGE = 10
REWARD_SMALL = -1
REWARD_FOR_BLOCK = 5

env = gym.envs.make("TTT-v0", small=REWARD_SMALL, large=REWARD_LARGE)

In [25]:
state_dict = create_state_dictionary()
state_size = len(state_dict.keys())
action_size = env.action_space.n

Number of legal states: 12092


In [26]:
# set training parameters
episodes = 960_000
max_steps = 9

In [27]:
exploration_parameters = {
    "max_epsilon": 1.0,
    "min_epsilon": 0.0,
    "decay_rate": 0.0001,
}

In [28]:
# Creating QLearning agent with learning params

lear_rate = 0.8
gamma = 0.9
qagent = QLearningAgent(exploration_parameters, state_size, action_size, learning_rate=lear_rate, gamma=gamma)

In [29]:
def check_for_potential_lose(state, color: int) -> bool:
        """check if after agent's move there is a lose position

        Args:
            color (int): of the player's enemy

        Returns:
            bool: indicating if this was a crucial move
        """
        state_check = np.copy(state)
        lose = False
        col = np.array([1,2])
        #enemy color
        enemy_color = color
        player_color = col[col != enemy_color][0]
        state_check[state_check == player_color] = -1
        state_check[state_check == enemy_color] = 1
        state_check = state_check.reshape(3,3)
        for ii in range(3):
            if (
                # check columns
                np.sum(state_check[:, ii]) == 2
                # check rows
                or np.sum(state_check[ii, :]) == 2
                # check diagonal
                or np.sum([state_check[0, 0], state_check[1, 1], state_check[2, 2]])
                == 2
                or np.sum([state_check[0, 2], state_check[1, 1], state_check[2, 0]])
                == 2
            ):
                lose = True
                break
        return lose

In [30]:
# Reverse dict where keys are int
reverse_dict = {value: key for key, value in state_dict.items()}

In [31]:
def play(qagent:QLearningAgent, player_color, state: int, action_space: np.array) -> tuple:
    action = qagent.get_action(state, action_space)

    pure_state = np.array(reverse_dict[state][:-1])
    # remove action from the action space
    action_space = action_space[action_space != action]
    # potential lose
    pot_lose = False
    col = np.array([1,2])
    if check_for_potential_lose(pure_state, col[col != player_color][0]):
        pot_lose = True

    new_state, reward, done, _ = env.step((action, player_color))

    # check for block opponent's win
    # if agent blocks opponnt's win agent get a reward
    if (done == False):
        if not check_for_potential_lose(new_state, col[col != player_color][0]) and pot_lose == True:
            reward += REWARD_FOR_BLOCK

    # Add player mark to a state
    new_state = np.append(new_state, player_color)
    new_state = state_dict[reshape_state(new_state)] 

    # Q-table update
    qagent.qtable[state, action] = qagent.update_qtable(
        state, new_state, action, reward, done
    )
    # new state
    state = new_state
    return state, action_space, done

In [32]:
def play_random(qagent:QLearningAgent, player_color, state: int, action_space: np.array) -> tuple:
    action = np.random.choice(action_space)
    action_space = action_space[action_space != action]
    new_state, reward, done, _ = env.step((action, player_color))
    new_state = np.append(new_state, player_color)
    new_state = state_dict[reshape_state(new_state)]
    state = new_state
    return state, action_space, done

In [33]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline
import random

start_time = time.time()

player_1 = Player(color=1, episodes=episodes)
player_2 = Player(color=2, episodes=episodes)

win_history = []

# best learning params
# Learning rate: 0.8, Win rate: 0.8, Gamma: 0.9

for episode in tqdm(range(episodes)):
    action_space = np.arange(9)

    # randomly change the order players
    start = np.random.choice([1,2])

    state, _ = env.reset()
    state = np.append(state, start)
    state = state_dict[reshape_state(state)]

    # One game
    for _step in range(start, max_steps + start):
        if _step == max_steps + start - 1:
            last_turn = True
        # change a turn
        if _step % 2 == 0:
            # state, action_space, done = play_random(qagent, player_1.color, state, action_space)
            state, action_space, done = play(qagent, player_1.color, state, action_space)
        else:
            state, action_space, done = play(qagent, player_2.color, state, action_space)
        if done == True:
            break

    # reduce epsilon for exporation-exploitation tradeoff
    qagent.update_epsilon(episode)

    #cur_win_rate, reward = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=100)

    #check how good is an agent
    if episode % 25_000 == 0:
        num_games = 50
        cur_win_rate, reward = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=num_games)
        win_history.append(sum(cur_win_rate)/num_games)
        print("WinRate:", sum(cur_win_rate)/num_games)
        # rewards.append(reward)
        # clear_output(True)
        # # plt.title('eps = {:e}, mean reward = {:.1f}'.format(agent.epsilon, np.mean(rewards[-10:])))
        # plt.plot(rewards)
        # plt.show()
    if episode % 25_000 == 0:

        sum_q_table = np.sum(qagent.qtable)
        time_passed = round((time.time() - start_time) / 60.0, 2)

        print(
            f"episode: {episode}, \
            epsilon: {round(qagent.epsilon, 2)}, \
            sum q-table: {sum_q_table}, \
            elapsed time [min]: {time_passed},  \
            done [%]: {episode / episodes * 100} \
            "
        )


  0%|          | 224/960000 [00:00<14:01, 1140.99it/s]

WinRate: 0.56
episode: 0,             epsilon: 1.0,             sum q-table: 38.46525335720089,             elapsed time [min]: 0.0,              done [%]: 0.0             


  3%|▎         | 25277/960000 [00:16<10:35, 1470.17it/s]

WinRate: 0.72
episode: 25000,             epsilon: 0.78,             sum q-table: 177219.86514620873,             elapsed time [min]: 0.28,              done [%]: 2.604166666666667             


  5%|▌         | 50257/960000 [00:33<10:07, 1497.16it/s]

...

WinRate: 0.8
episode: 625000,             epsilon: 0.0,             sum q-table: 235814.4112092051,             elapsed time [min]: 8.2,              done [%]: 65.10416666666666             


 68%|██████▊   | 650239/960000 [08:33<04:27, 1157.93it/s]

WinRate: 0.92
episode: 650000,             epsilon: 0.0,             sum q-table: 235814.4112092051,             elapsed time [min]: 8.55,              done [%]: 67.70833333333334             


 70%|██████▉   | 667393/960000 [08:47<03:51, 1264.82it/s]


KeyboardInterrupt: 

In [41]:
# Win Rate check

num_games = 1000
cur_win_rate, _ = play_tictactoe_with_random(env, qagent.qtable, state_dict, num_test_games=num_games)
win_history.append(sum(cur_win_rate)/num_games)
print("WinRate:", sum(cur_win_rate)/num_games)

WinRate: 0.862


In [42]:
qtable = qagent.qtable
save_qtable(qtable, 'tables', "q_table_best_check_for_block")

q_table_best_check_for_block.npy saved!


In [ ]:
qtable = load_qtable('tables', "q_table_best2")

In [47]:
#check how correct is q-table

state = np.random.choice(np.arange(env.observation_space.n))
# state_dict[state]
print(state)

key = list(filter(lambda x: state_dict[x] == state, state_dict))[0]
print(np.array(key[:-1]).reshape(3,3))
print("Turn was:", key[-1])
print(np.round(qagent.qtable[state].reshape(3,3),2))

# q = np.round(qtable[state],2)
# print("Action: ",np.argmax(q))

state_pure = np.array(key[:-1])
action_space = np.where(state_pure == 0)[0]

best_action = max(action_space, key=lambda action: qagent.qtable[state, action])
print(best_action)
# array = np.array(qtable[state, :])
# order = array.argsort()
# ranks = order.argsort()
# max_value_rank = np.min(ranks[action_space])
# action = np.where(ranks == max_value_rank)[0][0]
# action

6084
[[0 2 0]
 [1 1 0]
 [2 0 1]]
Turn was: 1
[[3.16 0.   7.02]
 [0.   0.   5.26]
 [0.   4.22 0.  ]]
2


In [165]:
play_tictactoe(env, qtable, state_dict, num_test_games=1)

Agent beginns
--------------------
╒═══╤═══╤═══╕
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
--------------------
move Agent
Action: 1
╒═══╤═══╤═══╕
│ - │ O │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛


--------------------
Move Human
Action: 0
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ - │ - │ - │
╘═══╧═══╧═══╛
-1


--------------------
move Agent
Action: 6
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ - │ - │ - │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛


--------------------
Move Human
Action: 5
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ - │ - │ X │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛
-1


--------------------
move Agent
Action: 3
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ O │ - │ X │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛


--------------------
Move Human
Action: 4
╒═══╤═══╤═══╕
│ X │ O │ - │
├───┼───┼───┤
│ O │ X │ X │
├───┼───┼───┤
│ O │ - │ - │
╘═══╧═══╧═══╛
-1


------